In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [46]:
# read excel file into a pandas dataframe

raw_df = pd.read_excel('../data/sarpol.xlsx')
raw_df.sort_values(['year', 'month', 'day', 'hour', 'min', 'sec'], inplace=True)
raw_df

,year,month,day,hour,min,sec,latitude,longitude,depth,magnitude
0,1900,2,24,0,30,0.0,38.450,44.870,33.0,5.40
1,1901,5,20,12,29,0.0,36.390,50.480,33.0,5.40
2,1902,9,5,4,33,0.0,39.500,48.000,33.0,5.40
3,1902,10,26,11,37,0.0,39.700,47.800,10.0,5.30
4,1903,2,9,5,18,0.0,36.580,47.650,200.0,5.60
...,...,...,...,...,...,...,...,...,...,...
26341,2022,1,9,19,19,51.1,39.186,46.588,10.0,3.17
26342,2022,1,10,18,15,8.0,35.637,44.984,10.0,3.75
26343,2022,1,10,18,17,40.7,36.985,49.675,18.0,3.00
26344,2022,1,10,18,29,49.2,35.619,44.949,10.0,4.67


In [48]:
MAX_MAG = raw_df['magnitude'].max()
MIN_MAG = raw_df['magnitude'].min()

print("Maximum magnitute: {}".format(MAX_MAG))
print("Minimum magnitute: {}".format(MIN_MAG))


Maximum magnitute: 7.6
Minimum magnitute: 2.8


In [49]:
# raw_df['date'] = pd.to_datetime(raw_df[['year', 'month', 'day']])
raw_df.set_index(['year', 'month'], inplace=True)
raw_df

day  hour  min   sec  latitude  longitude  depth  magnitude
year month                                                             
1900 2       24     0   30   0.0    38.450     44.870   33.0       5.40
1901 5       20    12   29   0.0    36.390     50.480   33.0       5.40
1902 9        5     4   33   0.0    39.500     48.000   33.0       5.40
     10      26    11   37   0.0    39.700     47.800   10.0       5.30
1903 2        9     5   18   0.0    36.580     47.650  200.0       5.60
...         ...   ...  ...   ...       ...        ...    ...        ...
2022 1        9    19   19  51.1    39.186     46.588   10.0       3.17
     1       10    18   15   8.0    35.637     44.984   10.0       3.75
     1       10    18   17  40.7    36.985     49.675   18.0       3.00
     1       10    18   29  49.2    35.619     44.949   10.0       4.67
     1       10    20   11  18.9    35.587     44.993   10.0       3.83

[26346 rows x 8 columns]

In [50]:
raw_df.isna().sum()

day          0
hour         0
min          0
sec          0
latitude     0
longitude    0
depth        0
magnitude    0
dtype: int64

In [51]:
# some hyperparameters for creating features

TIME_STEP = 50

## Easy method paper parameters

In [52]:
# extract the months with less than TIME_STEP rows

freq = raw_df.groupby(['year', 'month']).size()

freq = freq[freq < TIME_STEP]


# apply on raw_df
raw_df.drop(freq.index, inplace=True)
raw_df

day  hour  min    sec  latitude  longitude  depth  magnitude
year month                                                              
1976 11       1    20   31   0.00    39.700     43.400   58.0       3.95
     11       7    11    7  55.00    33.190     47.930  226.0       4.80
     11       7    11    7  58.00    33.200     47.940   25.0       5.63
     11      15     8    3  23.00    33.190     47.940  204.1       5.37
     11      20    22   40   0.00    38.290     46.000   62.0       3.95
...         ...   ...  ...    ...       ...        ...    ...        ...
2021 12      30    19   32  31.80    37.093     45.009    6.0       3.00
     12      30    21   31  28.90    39.872     45.624    7.0       3.34
     12      31     2   19   0.90    36.294     45.602    7.0       3.17
     12      31    18   27   0.54    32.301     49.789   10.0       3.09
     12      31    19   30  58.90    35.314     46.569    8.0       3.83

[20443 rows x 8 columns]

### Feature #1: T = Tn - T1

In [53]:
idx_list = set(raw_df.index.to_list())

# dataframe of features
df = pd.DataFrame()
# T = T_n - T_1
T_list = []
for i in idx_list:
    events = raw_df.loc[i] # the dataframe of events in a month
    time_diff = (events['day'].iloc[-1] - events['day'].iloc[-TIME_STEP]) * 86400 \
                + (events['hour'].iloc[-1] - events['hour'].iloc[-TIME_STEP]) * 3600 \
                + (events['min'].iloc[-1] - events['min'].iloc[-TIME_STEP]) * 60 \
                + (events['sec'].iloc[-1] - events['sec'].iloc[-TIME_STEP])
    
    time_diff /= (24 * 3600) # for converting to days
    T_list.append(time_diff)

    
df['T'] = T_list

In [54]:
df

,T
0,19.150396
1,18.433950
2,24.656961
3,23.524088
4,29.859480
...,...
182,18.933133
183,22.097975
184,23.602027
185,14.210067


### Feature #2: M_mean

In [55]:
M_mean_list = []
for i in idx_list:
    events = raw_df.loc[i] # the dataframe of events in a month
    M_mean_list.append(np.mean(events['magnitude'][-TIME_STEP:]))

    
df['M_mean'] = M_mean_list
df

,T,M_mean
0,19.150396,3.3264
1,18.433950,3.4754
2,24.656961,3.2694
3,23.524088,3.4956
4,29.859480,3.3408
...,...,...
182,18.933133,3.3936
183,22.097975,3.4566
184,23.602027,4.5096
185,14.210067,3.3848


### Feature #3: dE

In [60]:
dE_list = []
for i in idx_list:
    events = raw_df.loc[i] # the dataframe of events in a month
    dE_list.append(np.sum(events['magnitude'][-TIME_STEP:].apply(lambda x: np.sqrt(10 ** (7.1 + 2.06*x)))))
    
df['dE'] = dE_list / df['T']
df

,T,M_mean,dE
0,19.150396,3.3264,4.990014e+07
1,18.433950,3.4754,5.997897e+07
2,24.656961,3.2694,2.875335e+07
3,23.524088,3.4956,5.680462e+07
4,29.859480,3.3408,2.826382e+07
...,...,...,...
182,18.933133,3.3936,4.155233e+07
183,22.097975,3.4566,6.929064e+07
184,23.602027,4.5096,4.093904e+08
185,14.210067,3.3848,5.385801e+07


### Feature #4&5: M_expected & eta & delta_M

In [68]:
M_expected_list = []
eta_list = []
b_list = []
delta_M_list = []

for i in idx_list:
    events = raw_df.loc[i] # the dataframe of events in a month
    N = np.arange(1, TIME_STEP+1)
    mag = events['magnitude'][-TIME_STEP:]
    max_M = np.max(mag)
    

    hist_values = plt.hist(mag, bins=10, range=[MIN_MAG, MAX_MAG], cumulative=False, histtype='bar', log=True)
    
    plt.close()

    y = hist_values[0]
    X = []
    for idx in range(len(hist_values[1][:-1])):
        X.append(np.round((hist_values[1][idx] + hist_values[1][idx+1])/2, 2))
    idx_start = np.where(y != 0)[0][0]
    y = y[idx_start:]
    X = X[idx_start:]
    idx_end = np.where(y == 0)[0][0]
    y = np.log(y[:idx_end]) # log10
    X = X[:idx_end]
    X = np.array(X)
#     print(X)
#     print(y)
#     print()

    # apply linear regression
    lr = LinearRegression()

    lr.fit(X.reshape(-1, 1), y.reshape(-1, 1))
    a = lr.intercept_
    b = -1 * lr.coef_
    M_expected = a / (b + 0.000001)
    M_expected_list.append(M_expected[0][0])
    b_list.append(b[0][0])
    
    eta = 0
    eta = np.linalg.norm(y - lr.predict(X.reshape(-1, 1)))
    eta /= len(X)
    eta_list.append(eta)
    delta_M_list.append(np.abs(max_M - M_expected[0][0]))
    
    
df['M_expected'] = M_expected_list
df['eta'] = eta_list
df['b'] = b_list
df['delta_M'] = delta_M_list
df

,T,M_mean,dE,M_expected,eta,b,delta_M
0,19.150396,3.3264,4.990014e+07,4.734231,1.939324,1.978671,0.095769
1,18.433950,3.4754,5.997897e+07,5.126971,1.270954,1.614015,0.796971
2,24.656961,3.2694,2.875335e+07,4.423892,1.983661,2.585856,0.246108
3,23.524088,3.4956,5.680462e+07,5.833198,0.879417,1.125805,1.333198
4,29.859480,3.3408,2.826382e+07,4.774071,1.539937,1.973814,0.104071
...,...,...,...,...,...,...,...
182,18.933133,3.3936,4.155233e+07,6.043551,0.614767,1.072520,1.963551
183,22.097975,3.4566,6.929064e+07,5.154352,1.537565,1.585181,0.404352
184,23.602027,4.5096,4.093904e+08,2.167200,0.767589,-1.144388,2.762800
185,14.210067,3.3848,5.385801e+07,4.602087,1.776538,2.303172,0.182087


# Mu & C

In [69]:


for i in idx_list:
    events = raw_df.loc[i] # the dataframe of events in a month
    N = np.arange(1, TIME_STEP+1)
    mag = events['magnitude'][-TIME_STEP:]
    

    hist_values = plt.hist(mag, bins=10, range=[MIN_MAG, MAX_MAG], cumulative=False, histtype='bar', log=True)
    plt.close()

    y = hist_values[0]
    X = hist_values[1][:-1]
    idx_start = np.where(y != 0)[0][0]
    y = y[idx_start:]
    X = X[idx_start:]
    idx_end = np.where(y == 0)[0][0]
    y = y[:idx_end]
    X = X[:idx_end]
    X = np.array(X)
    
    print(X)
    print(y)
    print()

    
df

[2.8  3.28 3.76 4.24 4.72]
[28. 17.  3.  1.  1.]

[2.8  3.28 3.76 4.24]
[21. 18.  9.  2.]

[2.8  3.28 3.76 4.24]
[32. 15.  2.  1.]

[2.8  3.28 3.76 4.24]
[21. 18.  6.  5.]

[2.8  3.28 3.76 4.24]
[25. 20.  3.  2.]

[2.8  3.28 3.76 4.24]
[20. 24.  5.  1.]

[3.76 4.24 4.72 5.2 ]
[36.  5.  6.  3.]

[2.8  3.28 3.76 4.24]
[25. 18.  3.  4.]

[3.76 4.24 4.72 5.2 ]
[13. 28.  8.  1.]

[2.8  3.28 3.76 4.24]
[32. 11.  6.  1.]

[2.8  3.28 3.76 4.24 4.72]
[22. 17.  6.  4.  1.]

[2.8  3.28 3.76 4.24]
[20. 23.  4.  3.]

[2.8  3.28 3.76 4.24 4.72]
[31. 10.  5.  3.  1.]

[2.8  3.28 3.76 4.24 4.72]
[17. 24.  7.  1.  1.]

[2.8  3.28 3.76 4.24]
[25. 19.  5.  1.]

[2.8  3.28 3.76 4.24 4.72]
[19. 12. 12.  4.  3.]

[2.8  3.28 3.76]
[32. 16.  2.]

[2.8  3.28 3.76 4.24 4.72]
[30. 13.  5.  1.  1.]

[2.8  3.28 3.76]
[20. 24.  6.]

[2.8  3.28 3.76 4.24]
[27. 17.  5.  1.]

[2.8  3.28 3.76 4.24]
[22. 17.  8.  3.]

[2.8  3.28 3.76]
[23. 24.  3.]

[2.8  3.28 3.76 4.24]
[23. 21.  3.  3.]

[2.8  3.28 3.76 4.24]
[21. 20.

,T,M_mean,dE,M_expected,eta,b,delta_M
0,19.150396,3.3264,4.990014e+07,4.734231,1.939324,1.978671,0.095769
1,18.433950,3.4754,5.997897e+07,5.126971,1.270954,1.614015,0.796971
2,24.656961,3.2694,2.875335e+07,4.423892,1.983661,2.585856,0.246108
3,23.524088,3.4956,5.680462e+07,5.833198,0.879417,1.125805,1.333198
4,29.859480,3.3408,2.826382e+07,4.774071,1.539937,1.973814,0.104071
...,...,...,...,...,...,...,...
182,18.933133,3.3936,4.155233e+07,6.043551,0.614767,1.072520,1.963551
183,22.097975,3.4566,6.929064e+07,5.154352,1.537565,1.585181,0.404352
184,23.602027,4.5096,4.093904e+08,2.167200,0.767589,-1.144388,2.762800
185,14.210067,3.3848,5.385801e+07,4.602087,1.776538,2.303172,0.182087


## Label

In [77]:
label_list = []
for i in idx_list:
    events = raw_df.loc[i] # the dataframe of events in a month
    mag = events['magnitude'][-TIME_STEP:]
    flag = False
    for m in mag:
        if m >= 5:
            flag = True
    label_list.append(1 if flag else 0)
    
df['label'] = label_list
df

,T,M_mean,dE,M_expected,eta,b,delta_M,label
0,19.150396,3.3264,4.990014e+07,4.734231,1.939324,1.978671,0.095769,0
1,18.433950,3.4754,5.997897e+07,5.126971,1.270954,1.614015,0.796971,0
2,24.656961,3.2694,2.875335e+07,4.423892,1.983661,2.585856,0.246108,0
3,23.524088,3.4956,5.680462e+07,5.833198,0.879417,1.125805,1.333198,0
4,29.859480,3.3408,2.826382e+07,4.774071,1.539937,1.973814,0.104071,0
...,...,...,...,...,...,...,...,...
182,18.933133,3.3936,4.155233e+07,6.043551,0.614767,1.072520,1.963551,0
183,22.097975,3.4566,6.929064e+07,5.154352,1.537565,1.585181,0.404352,0
184,23.602027,4.5096,4.093904e+08,2.167200,0.767589,-1.144388,2.762800,0
185,14.210067,3.3848,5.385801e+07,4.602087,1.776538,2.303172,0.182087,0


In [78]:
len(df[df['label'] == 1])

42

In [79]:
X = df[['T', 'M_mean', 'dE', 'M_expected', 'eta', 'b', 'delta_M']]
y = df['label']

In [80]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

acc_score = []
k = 5
kf = KFold(n_splits=k)
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    
    model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    model.fit(X_train, y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
    
avg_acc_score = sum(acc_score)/k
print(avg_acc_score * 100)

80.73968705547652


In [81]:
df.isna().sum()

T             0
M_mean        0
dE            0
M_expected    0
eta           0
b             0
delta_M       0
label         0
dtype: int64